In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Kraus, SuperOp
from qiskit.visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator

# Import from Qiskit Aer noise module
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

from qiskit_aer import Aer
import qiskit
from qiskit import transpile

In [ ]:
def bb84_circuit(state, basis, measurement_basis):

    #state: array of 0s and 1s denoting the state to be encoded
    #basis: array of 0s and 1s denoting the basis to be used for encoding
                #0 -> Computational Basis
                #1 -> Hadamard Basis
    #meas_basis: array of 0s and 1s denoting the basis to be used for measurement
                #0 -> Computational Basis
                #1 -> Hadamard Basis

    num_qubits = len(state)

    circuit = QuantumCircuit(num_qubits)

    # Sender prepares qubits
    for i in range(len(basis)):
        if state[i] == 1:
            circuit.x(i)
        if basis[i] == 1:
            circuit.h(i)


    # Measuring action performed by Bob
    for i in range(len(measurement_basis)):
        if measurement_basis[i] == 1:
            circuit.h(i)


    circuit.measure_all()

    return circuit

In [ ]:
num_qubits=16
key_list=[]
noisy_key_list=[] #saving the keys from depolarizing channel
key_bit_flip_list=[] #saving the keys from bit-flip channel
alice_key_list=[] #Alice's keys; no noise

for  i in range(10):
    alice_basis = np.random.randint(2, size=num_qubits)
    alice_state = np.random.randint(2, size=num_qubits)
    bob_basis = np.random.randint(2, size=num_qubits)
    circuit = bb84_circuit(alice_state, alice_basis, bob_basis)
    backend = Aer.get_backend('qasm_simulator')

    circuit_new = transpile(circuit.reverse_bits(), backend=backend)
    job = backend.run(circuit_new, shots=1)
    key = job.result().get_counts().most_frequent()

    encryption_key = ''
    alice_key_errorless=''
    for i in range(num_qubits):
        if alice_basis[i] == bob_basis[i]:
             encryption_key += str(key[i])
             alice_key_errorless += str(alice_state[i])
    key_list.append(encryption_key)
    alice_key_list.append(alice_key_errorless)


    #INSERTING DEPOLARIZING ERROR
    #Create an empty noise model
    noise_model = NoiseModel()

    for i in range(len(alice_basis)):
        randerror=np.random.rand() #generating random error co-efficients
        error = depolarizing_error(randerror, 1)
        noise_model.add_quantum_error(error, ['rz', 'sx', 'u1', 'u2', 'u3'], [i]) #applying error to noise model


    # Print noise model info
    #print(noise_model)
    sim_noise = AerSimulator(noise_model=noise_model)

    passmanager = generate_preset_pass_manager(optimization_level=3, backend=sim_noise)
    circ_tnoise = passmanager.run(circuit)


    # Run and get counts
    result_bit_flip = sim_noise.run(circ_tnoise.reverse_bits(), shots=1).result()
    keys_bit_flip = result_bit_flip.get_counts(0).most_frequent()

    noisy_encryption_key = ''

    for i in range(num_qubits):
        if alice_basis[i] == bob_basis[i]:
             noisy_encryption_key += str(keys_bit_flip[i])

    noisy_key_list.append(noisy_encryption_key) #keys with depolarizing errors.



    # Add BIT FLIP errors to noise model
    noise_bit_flip = NoiseModel()
    for i in range(len(alice_basis)):
        randerror=np.random.rand()
        error_gate1 = pauli_error([('X',randerror), ('I', 1 - randerror)])
        noise_bit_flip.add_quantum_error(error_gate1, ['rz', 'sx',"u1", "u2", "u3"], [i])


    sim_noise = AerSimulator(noise_model=noise_bit_flip)
    passmanager = generate_preset_pass_manager(optimization_level=3, backend=sim_noise)
    circ_tnoise = passmanager.run(circuit)


    # Run and get counts
    result_bit_flip = sim_noise.run(circ_tnoise.reverse_bits(), shots=1).result()
    keys_bit_flip = result_bit_flip.get_counts(0).most_frequent()

    noisy_encryption_key = ''

    for i in range(num_qubits):
        if alice_basis[i] == bob_basis[i]:
             noisy_encryption_key += str(keys_bit_flip[i])


    key_bit_flip_list.append(noisy_encryption_key)


In [ ]:
def QBER(key1, key2):
    Lkey1=list(key1)
    Lkey2=list(key2)
    j=0
    for i in range(len(Lkey1)):
        if Lkey1[i]!= Lkey2[i]:
            j += 1
    return j/len(Lkey1)

Qber_depol = [QBER(i, j) for i, j in zip(alice_key_list, noisy_key_list)]
Qber_BP = [QBER(i, j) for i, j in zip(alice_key_list, key_bP_list)]

In [ ]:
#SAVING DATA AS TXT FLIE
import numpy as np
np.savetxt('Qber_Randdepol_0.dat', Qber_depol)
np.savetxt('Qber_Randbp_0.dat', Qber_BP)